In [15]:
import pandas as pd
import numpy as np

In [250]:
vt = pd.read_csv('vitals_trajectory.csv')

<ipython-input-250-64f1363dbd35>:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  vt = pd.read_csv('vitals_trajectory.csv')


## Filter  & standardization
MODS=1 on Day1, minute to hour and other cleaning

In [251]:
# drop completely redundant rows and keep MODS=day1 patients
all_sp = vt.drop('event_units',axis = 1).drop_duplicates()
mods_day1 = all_sp[all_sp['mods_day1']==1].drop('mods_day1',axis = 1)

In [252]:
# convert time from min to hour, reorder dataframe and clean age_group notation
mods_day1['event_hour'] = mods_day1['event_time'].div(60)
mods_day1['died_hour'] = mods_day1['died_time'].div(60*24)
mods_day1 = mods_day1.sort_values(by=['hospital_id','age_group','EncID','event_time','event_name'])
mods_day1['age_group'] = [i.split('.')[1] for i in mods_day1['age_group'].tolist()]
mods_day1

,EncID,hospital_id,age_group,mods_day7,died_in_hosp,died_time,event_name,event_time,event_value,event_hour,died_hour
377547,10236 L2_P01,P01,1 mo,0,0,NaN,PULSE,4.0,169.0,0.066667,NaN
645780,10236 L2_P01,P01,1 mo,0,0,NaN,PULSE_OX,4.0,99.0,0.066667,NaN
771672,10236 L2_P01,P01,1 mo,0,0,NaN,RESP_RATE,4.0,47.0,0.066667,NaN
995554,10236 L2_P01,P01,1 mo,0,0,NaN,TEMP,4.0,36.5,0.066667,NaN
377548,10236 L2_P01,P01,1 mo,0,0,NaN,PULSE,9.0,168.0,0.150000,NaN
...,...,...,...,...,...,...,...,...,...,...,...
5163515,990_P13,P13,144 to 216 mo,0,0,NaN,MAP,1434.0,68.0,23.900000,NaN
5057237,990_P13,P13,144 to 216 mo,0,0,NaN,PULSE,1434.0,96.0,23.900000,NaN
5299321,990_P13,P13,144 to 216 mo,0,0,NaN,PULSE_OX,1434.0,100.0,23.900000,NaN
5524450,990_P13,P13,144 to 216 mo,0,0,NaN,RESP_RATE,1434.0,26.0,23.900000,NaN


## Population characteristics

In [237]:
# number of MODS=1 on day 1 versus all patients
print(mods_day1['EncID'].nunique(), all_sp['EncID'].nunique())

12845 38398


In [238]:
# mods day 7 difference between mods on day1 and all patients
print(mods_day1.groupby('EncID').first()['mods_day7'].value_counts(normalize=True), all_sp.groupby('EncID').first()['mods_day7'].value_counts(normalize=True), sep='\n')

0    0.710315
1    0.289685
Name: mods_day7, dtype: float64
0    0.883119
1    0.116881
Name: mods_day7, dtype: float64


In [239]:
# death in hospital difference
print(mods_day1.groupby('EncID').first()['died_in_hosp'].value_counts(normalize=True), all_sp.groupby('EncID').first()['died_in_hosp'].value_counts(normalize=True), sep='\n')

0    0.892254
1    0.107746
Name: died_in_hosp, dtype: float64
0    0.953018
1    0.046982
Name: died_in_hosp, dtype: float64


In [240]:
# death in hospital count difference
print(mods_day1.groupby('EncID').first()['died_in_hosp'].value_counts(), all_sp.groupby('EncID').first()['died_in_hosp'].value_counts(), sep='\n')

0    11461
1     1384
Name: died_in_hosp, dtype: int64
0    36594
1     1804
Name: died_in_hosp, dtype: int64


In [241]:
# proportion in each group for mods day1
mods_day1.groupby('EncID').first()['age_group'].value_counts(normalize=True)

 60 to 143 mo     0.254885
 144 to 216 mo    0.242507
 2 to 11 mo       0.175710
 24 to 59 mo      0.170027
 12 to 24 mo      0.116388
 1 mo             0.040483
Name: age_group, dtype: float64

## Clean death time
replace negative values with last vital time

In [255]:
# replace negative death times with time of last vital
tmp = mods_day1.groupby('EncID').last()
last_event_time = tmp[tmp['died_time']<0][['event_time','event_hour']]
neg_death_id = last_event_time.index.tolist()
neg_death_time = last_event_time['event_time'].tolist()
neg_death_hr = last_event_time['event_hour'].tolist()
for idx,val in enumerate(neg_death):
    #print(mods_day1.loc[mods_day1.index ==val, 'died_time'])
    mods_day1.loc[mods_day1['EncID'] ==val, 'died_time'] = neg_death_time[idx]
    mods_day1.loc[mods_day1['EncID'] ==val, 'died_hour'] = neg_death_hr[idx]

In [243]:
mods_day1.groupby('EncID').first()['died_hour'].describe()

count    1520.000000
mean       65.801754
std       197.342752
min         0.000000
25%         2.000000
50%         6.635417
75%        31.105556
max      1888.763889
Name: died_hour, dtype: float64

In [256]:
mods_day1 = mods_day1.drop(['event_time','died_time'],axis = 1)

## Select first 12hr of vitals

In [257]:
#mods_day1.to_csv('mods_day1.csv',index=False)
mods_day1_12hr = mods_day1[mods_day1['event_hour']<13]
#mods_day1_12hr.to_csv('mods_day1_12hr.csv',index=False)

In [258]:
# available vitals in first 12 hours
mods_day1_12hr['event_name'].value_counts()

PULSE        346133
PULSE_OX     341325
MAP          340268
SBP          328342
RESP_RATE    314790
FIO2         168956
TEMP         134148
SF_RATIO     103497
Name: event_name, dtype: int64

## Re-calculate SF Ratio
Hourly SF ratio = hourly average of pulse_ox / hourly average of fio2

In [294]:
# re-derive sf_ratio
sf_ratio = mods_day1_12hr[mods_day1_12hr['event_name'].isin(['PULSE_OX','FIO2','SF_RATIO'])][['EncID','event_name','event_hour','event_value']]
sf_ratio['event_hr_floor'] = np.floor(sf_ratio['event_hour'])
pulse_ox = sf_ratio[sf_ratio['event_name'] == 'PULSE_OX']
fio2 = sf_ratio[sf_ratio['event_name'] == 'FIO2']
pulse_ox_ave = pd.DataFrame(pulse_ox.groupby(['EncID','event_hr_floor'])['event_value'].mean())
fio2_ave = pd.DataFrame(fio2.groupby(['EncID','event_hr_floor'])['event_value'].mean())

In [ ]:
# rederive sf_ratio
calc_sfratio = []
for i in pulse_ox_ave.index:
    encid = i[0]
    hr_floor = i[1]
    if (fio2_ave.index == i).any():
        fio2_row = fio2_ave.iloc[(fio2_ave.index.get_level_values('EncID') == encid) & (fio2_ave.index.get_level_values('event_hr_floor') == hr_floor)]
        fio2 = fio2_row['event_value'].tolist()[0]
        pulseox = pulse_ox_ave.loc[[i]]['event_value'].tolist()[0]
        calc = [encid, hr_floor, pulseox/fio2]
        calc_sfratio.append(calc)
        #print(calc)

In [376]:
# compare size of old sf_ratio and newly derived
# old sf ratio
sf_ratio_only = sf_ratio[sf_ratio['event_name'] == 'SF_RATIO']
sf_ratio_old_ave = pd.DataFrame(sf_ratio_only.groupby(['EncID','event_hr_floor'])['event_value'].mean())
print(sf_ratio_old_ave.shape[0], len(calc_sfratio))

59369 96044


In [374]:
# format newly derived sf_ratio to df
new_sfratio_df = pd.DataFrame(calc_sfratio)
new_sfratio_df.columns = ['EncID','event_hour','event_value']
new_sfratio_df['event_name'] = 'SF_RATIO'

In [375]:
new_sfratio_df

,EncID,event_hour,event_value,event_name
0,1000044335_P05,0.0,191.0,SF_RATIO
1,1000044335_P05,2.0,184.0,SF_RATIO
2,1000044335_P05,3.0,194.0,SF_RATIO
3,1000044335_P05,5.0,200.0,SF_RATIO
4,1000044335_P05,6.0,200.0,SF_RATIO
...,...,...,...,...
96039,999_P10,8.0,400.0,SF_RATIO
96040,999_P10,9.0,396.0,SF_RATIO
96041,999_P10,10.0,396.0,SF_RATIO
96042,999_P10,11.0,400.0,SF_RATIO


## Clean metadata of patients
remove duplicate death time

In [378]:
# extract metadata for each patient
meta = mods_day1_12hr[['EncID','hospital_id','age_group','mods_day7','died_in_hosp','died_hour']].drop_duplicates()

In [379]:
# check id of patients with multiple meta data
tmp = pd.DataFrame(meta['EncID'].value_counts())
dup_meta = tmp[tmp['EncID']>1].index.tolist()
meta_dup = meta[meta['EncID'].isin(dup_meta)]
# round to nearest thousandth and remove duplicate due to small time difference
meta_dup['died_hour_round'] = round(meta_dup['died_hour'],2)
meta_dup = meta_dup.drop('died_hour',axis = 1)
meta_dup = meta_dup.drop_duplicates()
# no more dup
meta_dup['EncID'].value_counts()

<ipython-input-379-ccc85edddd1c>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  meta_dup['died_hour_round'] = round(meta_dup['died_hour'],2)


17111_P02    1
12687_P02    1
19992_P02    1
24390_P02    1
24510_P02    1
            ..
37113_P02    1
40197_P02    1
5309_P02     1
8600_P02     1
8583_P02     1
Name: EncID, Length: 65, dtype: int64

In [380]:
meta['died_hour_round'] = round(meta['died_hour'], 2)
meta = meta.drop('died_hour', axis = 1)
meta = meta.drop_duplicates()

In [381]:
meta

,EncID,hospital_id,age_group,mods_day7,died_in_hosp,died_hour_round
377547,10236 L2_P01,P01,1 mo,0,0,NaN
547648,12801 L2_P01,P01,1 mo,0,0,NaN
552534,13654 L2_P01,P01,1 mo,0,0,NaN
552975,13681 L2_P01,P01,1 mo,1,1,4.0
465088,14435 L2_P01,P01,1 mo,1,1,1.0
...,...,...,...,...,...,...
5387151,964_P13,P13,144 to 216 mo,0,0,NaN
5162978,96_P13,P13,144 to 216 mo,0,0,NaN
5399799,97_P13,P13,144 to 216 mo,0,0,NaN
5387387,984_P13,P13,144 to 216 mo,0,0,NaN


## Calculate hourly average for all vitals
replace vitals that have multiple measurements in an hour with hourly average

In [421]:
# extract vitals only dataframe
vitals = mods_day1_12hr[['EncID','event_name','event_hour','event_value']]
vitals['event_hr_floor'] = np.floor(vitals['event_hour'])  
             
# helper for calculating hourly average
def hr_ave(vit):
    df = vitals[vitals['event_name'] == vit]
    df = pd.DataFrame(df.groupby(['EncID','event_hr_floor'])['event_value'].mean())
    df['EncID'] = [i[0] for i in df.index.tolist()]
    df['event_hour'] = [i[1] for i in df.index.tolist()]
    df = df.reset_index(drop=True)
    df = df[['EncID', 'event_hour','event_value']]
    df['event_name'] = vit
    return df

<ipython-input-421-0bc9261d334b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vitals['event_hr_floor'] = np.floor(vitals['event_hour'])


In [454]:
# calculate hourly mean for all vitals PULSE, MAP, SBP, RESP_RATE, TEMP
pulse_ave = hr_ave('PULSE')
map_ave = hr_ave('MAP')
sbp_ave = hr_ave('SBP')
resp_ave = hr_ave('RESP_RATE')
temp_ave = hr_ave('TEMP')
fio2_ave = hr_ave('FIO2')
pulseox_ave = hr_ave('PULSE_OX')

In [455]:
print(pulse_ave.shape, map_ave.shape, sbp_ave.shape, resp_ave.shape, temp_ave.shape,fio2_ave.shape,pulseox_ave.shape)

(154256, 4) (141212, 4) (141921, 4) (150122, 4) (94331, 4) (97637, 4) (153359, 4)


## Merge new vitals with metadata

In [456]:
# concatenation of vitals
tmp = pd.concat([pulse_ave, map_ave], axis=0)
tmp = pd.concat([tmp, sbp_ave], axis=0)
tmp = pd.concat([tmp, resp_ave], axis=0)
tmp = pd.concat([tmp, temp_ave], axis=0)
tmp = pd.concat([tmp, fio2_ave], axis=0)
tmp = pd.concat([tmp, pulseox_ave], axis=0)
vital_hrave = pd.concat([tmp, new_sfratio_df], axis=0)

In [460]:
# merge with meta data
new_df = vital_hrave.merge(meta, on='EncID', how='left')
new_df = new_df.sort_values(by=['hospital_id','age_group','EncID','event_hour','event_name'])

In [461]:
new_df.to_csv('vitals_trajectory_cleaned.csv',index=False)

## Explore reasons of missing data 

In [459]:
new_df[(new_df['event_name']=='TEMP') & (new_df['event_hour']==2)]['EncID'].nunique()

6938

In [452]:
new_df[['EncID','hospital_id']].drop_duplicates()['hospital_id'].value_counts()

P04    1772
P13    1689
P05    1471
P12    1254
P07    1121
P11    1041
P01     962
P02     779
P10     770
P03     690
P06     655
P09     486
P08     151
Name: hospital_id, dtype: int64

In [447]:
# for temp at hour 2, only 6938 patients have data, but not due to hospital difference
new_df[(new_df['event_name']=='TEMP') & (new_df['event_hour']==2)]['hospital_id'].value_counts()

P04    1121
P05     986
P13     808
P12     715
P07     641
P01     504
P11     423
P03     414
P02     409
P06     333
P10     294
P09     187
P08     103
Name: hospital_id, dtype: int64

In [448]:
# also not due to age
new_df[(new_df['event_name']=='TEMP') & (new_df['event_hour']==2)]['age_group'].value_counts()

 60 to 143 mo     1749
 144 to 216 mo    1629
 2 to 11 mo       1282
 24 to 59 mo      1178
 12 to 24 mo       788
 1 mo              312
Name: age_group, dtype: int64